#### **Client Configuration**

In [1]:
# Importing the 'nbimporter' package which allows you to import Jupyter notebooks as Python modules
import nbimporter

# Commented out line (doesn't execute): The '%run' magic command would run the '01_grokai_chat_intro.ipynb' notebook in Jupyter.
## %run 01_grokai_chat_intro.ipynb

# Importing 'sys' to interact with the Python runtime environment (for modifying the system path)
import sys

# Importing 'os' to work with the operating system, like handling file paths
import os

# Add the absolute path to the project directory ('genai_project') to the Python path
# This allows Python to find and import modules from this directory
sys.path.append(os.path.abspath("C:/Users/dhira/Desktop/genai_project"))

# Import the 'client' object from the 'grokai_client_setup.py' file located in the project directory
# This client is likely responsible for setting up communication with the GrokAI system
from grokai_client_setup import client

✅ Groq client configured successfully and ready to use.


#### **🎯 Topic 1 — Why LLM Parameters Matter**

Large Language Models (LLMs) are inherently **probabilistic**:
- This means the output varies based on input parameters
- If you use default settings (temperature=1, max_tokens=2048), the model’s behavior will be **random**, **unpredictable**, and **costly** in production.

Understanding **LLM parameters** is **critical** for:
- Reducing hallucinations
- Controlling cost
- Making your models more deterministic and predictable

**In this topic, we will:**
- Understand what parameters control
- Discuss how to fine-tune them for specific use cases
- Explore common mistakes and best practices

#### **🔍 Why Do LLMs Have Parameters?**

LLMs are designed to **generate text** based on probability.
- Without parameters, the model would give random responses.
- Parameters allow us to **shape** this randomness for desired behaviors.

For instance:
- **temperature** controls how creative or deterministic the model is.
- **top_p** limits token selection to the top N most probable tokens.
- **max_tokens** restricts how long the response can be.
- **stop** sequences ensure responses are cut off cleanly.

These controls let you **fine-tune** the model’s outputs for **specific tasks**.

### Why we must **not rely on default settings**:
- Default values are **good for demos** but not for real-world applications.
- In real-world applications, **controlled behavior** is paramount for **reliability**.


#### **🧠 Senior-Level Insight**

> "Without parameter control, you are at the mercy of the model's randomness."

In production:
- **LLM behavior


#### **🎯 Topic 2 — Temperature (Creativity vs Determinism)**

Temperature is one of the most important **LLM parameters** that controls the **creativity** vs **determinism** of the model's output.

### What is Temperature?
- **Temperature** ranges from 0 to 1 (or higher in some cases).
- At **high temperatures (e.g., 0.8)**, the model’s output is **more creative** and **less deterministic**.
- At **low temperatures (e.g., 0.2)**, the model's output is **more predictable**, **conservative**, and **consistent**.

### Why It Matters:
- **High temperature**: Increases randomness. Useful for tasks that need creativity, like writing poetry or brainstorming.
- **Low temperature**: Reduces randomness. Ideal for factual answers, like generating documentation, code completion, or providing precise information.

For example:
- **Temperature = 0.0** → Extremely deterministic, always similar responses.
- **Temperature = 1.0** → Fully creative, with high variability.

#### **Use Temperature to Control Output Creativity**

In [2]:
# ============================================================
# 📘 SECTION 1 — Set Low Temperature (Deterministic)
# ------------------------------------------------------------
# Why?
#   - This example demonstrates a low temperature, forcing the model to give more predictable, deterministic responses.
#   - Ideal for structured or fact-based outputs.
# ============================================================

prompt = "What is capital of france?"

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",  # Select model
    messages=[{"role":"user","content":prompt}],
    temperature=0.2, # Low temperature for deterministic output
    max_tokens=100
)

# Extract and display the response
assistant_reply = response.choices[0].message.content
print("Low Temperature Response (Predictable):")
print(assistant_reply)

# ============================================================
# 📘 SECTION 2 — Set High Temperature (Creative)
# ------------------------------------------------------------
# Why?
#   - This example demonstrates a high temperature, allowing for creative and varied responses.
#   - Ideal for unstructured, creative tasks like storytelling or generating ideas.
# ============================================================

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user","content":prompt}],
    temperature=0.8, # High temperature for creative output
    max_tokens=100
)

# Extract and display the response
assistant_reply = response.choices[0].message.content

print("\nHigh Temperature Response (Creative):")
print(assistant_reply)

Low Temperature Response (Predictable):
The capital of France is Paris.

High Temperature Response (Creative):
The capital of France is Paris.


#### **Explanation:**

- In **Section 1**, we're setting **temperature=0.2**. This ensures that the model gives **consistent and predictable responses**. 
   - Example: **What is the capital of France?** → Answer: "Paris."
   - Very low variability, always the same or similar answer.
  
- In **Section 2**, we increase **temperature to 0.8**. This introduces more **creativity** in the model’s responses.
   - Example: The same question about France could get more imaginative answers with varied phrasing and unexpected details.

The **temperature parameter** essentially determines how **creative** or **controlled** the model will be when answering.

In real-world scenarios:
- **Low temperature** is preferred when you need **precise answers**.
- **High temperature** is used when you need the model to **generate ideas** or be **creative**.


#### **Temperature Tuning in Production**

When working with GenAI in production, **temperature tuning** is essential for controlling the system's output.

- **High temperature** could lead to **hallucinations** or **irrelevant answers**, which might cause **loss of user trust**.
- **Low temperature** ensures that answers are **predictable**, but if set too low, it could lack **creativity** and **engagement**.

Best practice:
- **Use low temperatures** for factual, structured tasks (e.g., data extraction).
- **Use higher temperatures** for exploratory tasks (e.g., content generation, idea brainstorming).


**✅ Topic 2 Final Summary (Rule Applied)**

Temperature controls the randomness of LLM responses

Low temperature = more deterministic answers

High temperature = more creative and varied answers

Production Impact: The wrong temperature can lead to inefficient or inaccurate responses.

#### **🎯 Topic 3 — top_p (Nucleus Sampling)**

top_p, also known as **Nucleus Sampling**, is a parameter used to control the **quality** and **diversity** of the model's output.

#### **What is top_p?**
- **top_p** controls the **cumulative probability** of token selection.
- Instead of selecting the **highest probability token**, top-p uses **cumulative probability** to choose the top "p" most likely tokens, ensuring diversity.

#### **Why top_p Matters:**
- It’s useful for controlling **creativity** while still staying within a certain **probability range**.
- **top_p=0.9** means: the model will choose from the top 90% of tokens (excluding the least likely ones).

#### **When to Use top_p vs Temperature:**
- **top_p** limits the number of tokens sampled, which makes it different from **temperature**.
- **Temperature** affects randomness; **top_p** affects how much to sample from the "nucleus" of likely tokens.

It can reduce the chances of the model generating **irrelevant or low-quality** output.

#### **Using top_p to Control Output Quality**

In [3]:
# ============================================================
# 📘 SECTION 1 — Set Low top_p (Nucleus Sampling)
# ------------------------------------------------------------
# Why?
#   - This example demonstrates a low top_p (e.g., 0.2)
#   - It forces the model to choose from a very **small** set of possible tokens, ensuring **high predictability** in output.
# ============================================================

prompt = "Explain the difference between a tuple and a list in Python."

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user","content":prompt}],
    temperature=0.7,
    top_p=0.2,
    max_tokens=100
)

assistant_reply = response.choices[0].message.content

print("Low top_p Response (Predictable):")
print(assistant_reply)

# ============================================================
# 📘 SECTION 2 — Set High top_p (Nucleus Sampling)
# ------------------------------------------------------------
# Why?
#   - This example demonstrates a high top_p (e.g., 0.9), increasing the range of possible tokens.
#   - This increases **creativity** and generates more **diverse** responses.
# ============================================================

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user","content":prompt}],
    temperature=0.7,
    top_p=0.9, # High top_p for creative output
    max_tokens=100
)

assistant_reply = response.choices[0].message.content
print("\nHigh top_p Response (Creative):")
print(assistant_reply)

Low top_p Response (Predictable):
**Tuples vs Lists in Python**

In Python, `tuples` and `lists` are two fundamental data structures that can store multiple values. While they share some similarities, there are key differences between them.

**Lists**
---------

A `list` is a mutable, ordered collection of items that can be of any data type, including strings, integers, floats, and other lists. Lists are denoted by square brackets `[]` and elements are separated by commas.

**Example

High top_p Response (Creative):
**Tuples vs Lists in Python**

In Python, `tuples` and `lists` are two types of data structures that can store multiple values. While they share some similarities, they have distinct differences in terms of their usage, characteristics, and behavior.

**Lists**
---------

A `list` is a mutable collection of items that can be of any data type, including strings, integers, floats, and other lists. Lists are denoted by square brackets `[]` and are defined


#### **Explanation:**

- **In Section 1**, we set **top_p=0.2**, which means the model will choose from a very narrow range of likely tokens, ensuring **high predictability** in responses.
  - Example: **“Explain the difference between a tuple and a list in Python”** → The response will be **concise and factual**.

- **In Section 2**, we increase **top_p to 0.9**, allowing the model to sample from a much wider range of possible tokens. This increases **creativity** and can lead to more **varied responses**.
  - Example: The same question could now yield more **creative** answers, with additional elaboration or different phrasing.

The **top_p parameter** essentially controls the **breadth of sampling**, helping strike a balance between **creativity** and **accuracy**.


#### **top_p in Production**

When you need **diverse** outputs for tasks like:
- brainstorming ideas
- creative writing
- generating varied responses

**High top_p values** (0.8–1.0) are appropriate.

For **factual consistency** and **structure** in tasks like:
- documentation
- code completion
- structured answers

**Low top_p values** (0.2–0.3) are ideal to reduce randomness and ensure more predictable results.

#### **✅ Topic 3 Final Summary (Rule Applied)**

top_p determines the diversity of the model’s output.

Low top_p (0.2): Restricts the model to a narrow, deterministic set of tokens.

High top_p (0.9): Increases creativity by sampling from a broader range of tokens.

Use top_p to fine-tune the balance between randomness and predictability.

#### **🎯 Topic 4 — max_tokens (Cost & Safety Guardrail)**

`max_tokens` controls the **maximum length** of the model’s output.

Why this matters in real systems:
- **Cost control:** more tokens = more cost
- **Latency control:** longer output = slower response
- **Safety control:** prevents the model from producing very long, unnecessary, or risky outputs

Key idea:
If you don’t set `max_tokens`, your system can become:
- expensive
- slow
- unpredictable

#### **max_tokens in Action (Low vs High)**

In [4]:
# ============================================================
# 📘 SECTION 1 — Prepare a Single Prompt
# ------------------------------------------------------------
# Why?
#   - Same prompt for both tests so we can compare max_tokens fairly
# ============================================================

prompt = "Explain Python tuples with an example."

# ============================================================
# 📘 SECTION 2 — Low max_tokens (Short + Controlled Output)
# ------------------------------------------------------------
# Why?
#   - Forces a short answer
#   - Useful for UI responses, summaries, and cost control
# ============================================================

response_low = client.chat.completions.create(
    model="llama-3.1-8b-instant",                 # Using our course-standard model
    messages=[{"role":"user","content":prompt}],
    temperature=0.3,            # Low creativity to keep response stable
    max_tokens=30               # ✅ LIMIT output length
)

reply_low = response_low.choices[0].message.content
print("===== ✅ Low max_tokens (30) =====")
print(reply_low)


# ============================================================
# 📘 SECTION 3 — Higher max_tokens (More Detailed Output)
# ------------------------------------------------------------
# Why?
#   - Allows longer explanation
#   - Useful for tutorials, deep explanations, documentation-like responses
# ============================================================


response_high = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3,
    max_tokens=200                                # ✅ ALLOW more detail
)

reply_high = response_high.choices[0].message.content
print("\n===== ✅ Higher max_tokens (200) =====")
print(reply_high)

===== ✅ Low max_tokens (30) =====
**Python Tuples**

Tuples in Python are immutable collections of objects that can be of any data type, including strings, integers, floats

===== ✅ Higher max_tokens (200) =====
**Python Tuples**

A tuple in Python is a collection of objects that can be of any data type, including strings, integers, floats, and other tuples. Tuples are defined by enclosing the values in parentheses `()` and are immutable, meaning they cannot be changed after creation.

**Example**
-----------

Here's an example of creating a tuple in Python:

```python
# Create a tuple
my_tuple = ("apple", "banana", "cherry")

# Accessing tuple elements
print(my_tuple[0])  # Output: apple
print(my_tuple[1])  # Output: banana
print(my_tuple[2])  # Output: cherry

# Tuple indexing
print(my_tuple[0:2])  # Output: ('apple', 'banana')

# Tuple unpacking
fruit1, fruit2, fruit3 = my_tuple
print(fruit1)  # Output: apple
print(fruit2)  # Output: banana
print(f


#### **🔍 What to Observe**

When max_tokens is LOW:
- the answer may feel incomplete
- model may stop mid-explanation
- output is cheap + fast

When max_tokens is HIGH:
- the answer is more complete
- higher cost + slightly slower
- better for “teaching mode”


#### **🧠 How Senior Engineers Use max_tokens**

#### ✅ Chatbots (fast UX)
- max_tokens: 100–300
- goal: quick, concise answers

#### ✅ RAG answers (grounded + focused)
- max_tokens: 200–500
- goal: answer + citations/snippets

#### ✅ Extraction / JSON tasks
- max_tokens: small and strict (50–200)
- goal: prevent extra text

#### ✅ Long-form content generation
- max_tokens: 800+
- goal: detailed content, but with cost guardrails


**✅ Topic 4 Final Summary**

max_tokens is a hard safety + cost limiter

Low values = fast + cheap but may truncate

High values = richer output but higher cost/latency

Production systems ALWAYS set max_tokens intentionally

#### **🎯 Topic 5 — Stop Sequences (Output Control)**

A **stop sequence** tells the LLM:
“Stop generating text when you reach THIS token or pattern.”

Why this matters:
- Prevents extra text
- Prevents explanations when you only want data
- Critical for JSON, APIs, agents, and tools

Stop sequences are a **hard boundary**, unlike temperature or top_p.

#### **❌ The Problem Without Stop Sequences**

LLMs like to be helpful.
That means they often add:
- explanations
- comments
- extra text

This breaks:
- APIs
- JSON parsing
- downstream systems

We must explicitly tell the model **where to stop**.

#### **Without Stop Sequence (Observe the Problem)**

In [5]:
# ============================================================
# 📘 SECTION 1 — Request WITHOUT Stop Sequence
# ------------------------------------------------------------
# Why?
#   - To observe how the model may add extra text
# ============================================================

prompt = """
Return ONLY the user's name and age as JSON.
"""

response_no_stop = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role":"system","content":"You are a strict JSON generator."},
        {"role":"user","content":prompt}],
    temperature=0.3,
    max_tokens=100
)

reply_no_stop = response_no_stop.choices[0].message.content

print("===== ❌ Without Stop Sequence =====")
print(reply_no_stop)

===== ❌ Without Stop Sequence =====
```json
{
  "name": "John Doe",
  "age": 30
}
```

Note: I assume the user's name is "John Doe" and their age is 30. You can replace these values as per your requirement.


#### **What You May See**

Even with strict instructions, the model may return:

- Extra explanations
- Text before or after JSON
- Markdown formatting

This is NORMAL LLM behavior — not a bug.

#### **WITH Stop Sequence**


In [6]:
# ============================================================
# 📘 SECTION 2 — Request WITH Stop Sequence
# ------------------------------------------------------------
# Why?
#   - Force the model to stop generation exactly where we want
#   - Prevent extra text beyond JSON
# ============================================================

response_with_stop = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role":"system","content":"You are strict JSON generator."},
        {"role":"user","content":prompt}],
    temperature=0.3,
    max_tokens=100,
    stop=["}"] # 🔴 Stop generation when JSON closes
)

reply_with_stop = response_with_stop.choices[0].message.content + "}"

print("\n===== ✅ With Stop Sequence =====")
print(reply_with_stop)


===== ✅ With Stop Sequence =====
```json
{
  "name": "John Doe",
  "age": 30
}


#### **🧠 Senior Insight — When Stop Sequences Are Mandatory**

Always use stop sequences when:
- Returning JSON
- Returning SQL
- Returning code snippets
- Returning tool arguments
- Feeding output into another system

If you don’t:
- Parsing WILL break
- Bugs WILL appear
- Systems WILL fail silently

#### **✅ Topic 5 Summary**

- Stop sequences are **hard output boundaries**
- They prevent extra text and hallucinations
- Essential for APIs, agents, tools, and RAG
- One of the most underrated production controls

#### **🎯 Topic 6 — Parameter Combinations & Safe Defaults**

In real systems, parameters are **never used in isolation**.

A senior GenAI engineer thinks in terms of:
- use case
- user experience
- cost
- reliability
- safety

This topic teaches **how parameters work together** and
what safe defaults look like in production.

#### **🧠 Core Idea**

Each parameter controls ONE dimension:
- temperature → randomness
- top_p → token diversity
- max_tokens → length & cost
- stop → hard output boundary

Correct behavior emerges from **balanced combinations**,
not extreme values.


#### **📘 Chatbot (User-Facing, Conversational)**

In [12]:
# ============================================================
# 📘 PRESET 1 — Chatbot (User-Facing, Conversational)
# ------------------------------------------------------------
# Why these values?
#   - temperature: slight creativity for natural tone
#   - top_p: controlled diversity
#   - max_tokens: prevent long, costly replies
#   - stop: not needed for free-form chat
# ============================================================

response_chatbot = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user","content":"Explain python list simply."}],
    temperature=0.6,
    top_p=0.9,
    max_tokens=200
)

print("🤖 Chatbot Response:")
print(response_chatbot.choices[0].message.content)

🤖 Chatbot Response:
**Python List Overview**

In Python, a list is a collection of items that can be of any data type, including strings, integers, floats, and other lists. Lists are denoted by square brackets `[]` and are ordered, meaning that items have a specific position or index.

**Basic List Operations**
-------------------------

### Creating a List

```python
my_list = [1, 2, 3, 4, 5]
```

### Accessing List Elements

```python
print(my_list[0])  # Output: 1
print(my_list[-1])  # Output: 5 (accesses the last element)
```

### Modifying List Elements

```python
my_list[0] = 10
print(my_list)  # Output: [10, 2, 3, 4, 5]
```

### Adding Elements to a List

```python
my_list.append(6)
print(my


#### **📘 API / Structured Output**

In [18]:
# ============================================================
# 📘 PRESET 2 — API / Structured Output
# ------------------------------------------------------------
# Why these values?
#   - temperature: low → deterministic
#   - top_p: low → limit randomness
#   - max_tokens: strict → cost + safety
#   - stop: mandatory → prevent extra text
# ============================================================

response_api = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role":"system","content":"Return ONLY JSON."},
        {"role":"user","content":"Return user age as JSON"}],
    temperature=0.2,
    top_p=0.3,
    max_tokens=50,
    stop=["}"]
)

json_output = response_api.choices[0].message.content + "}"
print("📦 API Output:")
print(json_output)

📦 API Output:
```python
import json

def get_user_age():
    user_age = 30  # Replace with actual user age
    return json.dumps({"user_age": user_age}


#### **RAG Answer (Grounded, Focused)**

In [ ]:
# ============================================================
# 📘 PRESET 3 — RAG Answer (Grounded + Controlled)
# ------------------------------------------------------------
# Why these values?
#   - temperature: low → reduce hallucinations
#   - top_p: moderate → readable answers
#   - max_tokens: enough for explanation, not rambling
# ============================================================
response_rag = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role":"system","content":"Answer ONLY using provided context."},
        {"role":"user","content":"What is a Python tuple?"}],
    temperature=0.3,
    top_p=0.6,
    max_tokens=300
)
reply_response_rag = response_rag.choices[0].message.content

print("📚 RAG-Style Response:")
print(reply_response_rag)

📚 RAG-Style Response:
In Python, a tuple is an immutable collection of objects. It is similar to a list, but unlike lists, tuples cannot be modified after they are created. Tuples are defined by enclosing a sequence of values in parentheses `()`.

Here's an example of a tuple:

```python
my_tuple = (1, 2, 3, 4, 5)
```

Tuples are often used when you need to store a collection of values that shouldn't be changed. They are also faster and more memory-efficient than lists because they are immutable.

Some key characteristics of tuples include:

- They are immutable, meaning their contents cannot be modified after creation.
- They are defined using parentheses `()`.
- They can contain any type of object, including strings, integers, floats, and other tuples.
- They support indexing and slicing, just like lists.
- They support methods like `len()`, `index()`, and `count()`, but not `append()`, `insert()`, or `remove()`.


#### **✅ Safe Defaults Cheat Sheet**

#### Chatbots
- temperature: 0.5–0.7
- top_p: 0.8–0.9
- max_tokens: 150–300

#### APIs / JSON / Tools
- temperature: 0.0–0.3
- top_p: 0.2–0.4
- max_tokens: 50–200
- stop: REQUIRED

#### RAG Systems
- temperature: 0.2–0.4
- top_p: 0.5–0.7
- max_tokens: 200–500


#### **🧠 Interview Insight**

Question:
“How do you reduce hallucinations in production?”

Strong answer:
“By combining low temperature, controlled top_p,
strict max_tokens, stop sequences, and grounded prompts —
not by relying on a single parameter.”


#### **✅ Topic 6 Summary**

- Parameters must be tuned **together**
- Safe defaults depend on use case
- Production systems never rely on defaults
- This is where junior engineers usually fail

#### **🧪 Topic 7 Micro Practice: Change Parameters & Observe (Hands-on)**

#### Goal — Build Intuition (Not Memorization)

The goal here is NOT to memorize values.

It is to:
- change ONE parameter at a time
- observe the output difference
- understand *why* the behavior changed

This is how senior engineers learn.

#### **Practice 1: Temperature Only**

In [34]:
# ============================================================
# 📘 PRACTICE 1 — Change Temperature Only
# ------------------------------------------------------------
# Rule:
#   - Keep everything SAME
#   - Change ONLY temperature
# ============================================================

prompt = "Explain Python dictionaries in simple terms."

# Low temperature (deterministic)
response_low_temp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2,      # 🔽 Low randomness
    top_p=0.9,
    max_tokens=150
)

print("===== Low Temperature (0.2) =====")
print(response_low_temp.choices[0].message.content)

# High temperature (creative)
response_high_temp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.8,      # 🔼 High randomness
    top_p=0.9,
    max_tokens=150
)

print("\n===== High Temperature (0.8) =====")
print(response_high_temp.choices[0].message.content)

===== Low Temperature (0.2) =====
**What are Python Dictionaries?**

In Python, a dictionary is a data structure that stores a collection of key-value pairs. It's like a phonebook where you have names (keys) and phone numbers (values).

**Key Features:**

1. **Keys**: These are unique identifiers for each value. They can be strings, numbers, or even other data types.
2. **Values**: These are the actual data stored in the dictionary. They can be strings, numbers, lists, dictionaries, or any other data type.
3. **Unordered**: Dictionaries don't maintain a specific order of their items.
4. **Mutable**: Dictionaries can be modified after they're created.

**How to Create a Dictionary:**

You

===== High Temperature (0.8) =====
**What is a Dictionary in Python?**

In Python, a dictionary is a collection of key-value pairs. Think of it like a phonebook where you store names (keys) and phone numbers (values).

**Key Features:**

1. **Keys:** These are the names or identifiers of the values. T

#### **📘 PRACTICE 2 — Change top_p Only**

In [35]:
# ============================================================
# 📘 PRACTICE 2 — Change top_p Only
# ------------------------------------------------------------
# Rule:
#   - Keep temperature SAME
#   - Change ONLY top_p
# ============================================================

# Narrow nucleus
response_low_top_p = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
    top_p=0.2,            # 🔽 Narrow token pool
    max_tokens=150
)

print("===== Low top_p (0.2) =====")
print(response_low_top_p.choices[0].message.content)


# Wide nucleus
response_high_top_p = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
    top_p=0.9,            # 🔼 Wider token pool
    max_tokens=150
)

print("\n===== High top_p (0.9) =====")
print(response_high_top_p.choices[0].message.content)


===== Low top_p (0.2) =====
**What are Python Dictionaries?**

In Python, a dictionary is a data structure that stores a collection of key-value pairs. It's like a phonebook where you have names (keys) and phone numbers (values).

**Key Features:**

1. **Keys**: These are unique identifiers for each value in the dictionary. They can be strings, numbers, or even other data types.
2. **Values**: These are the actual data stored in the dictionary, associated with each key.
3. **Unordered**: Dictionaries don't have a specific order, unlike lists or arrays.
4. **Mutable**: You can add, remove, or modify key-value pairs in a dictionary.

**Example:**

```python
# Create a dictionary

===== High top_p (0.9) =====
**What are Python Dictionaries?**

In Python, a dictionary is a data structure that stores a collection of key-value pairs. It's like a phonebook where you have names (keys) and phone numbers (values).

**Key Features:**

1. **Key-Value Pairs:** Dictionaries consist of key-value pair

#### **🔍 What to Observe**

Low max_tokens → truncated or brief answers

High max_tokens → fuller explanations

#### **🧠 My Observations**

- Temperature controls **creativity**
- top_p controls **token diversity**
- max_tokens controls **length & cost**
- Changing ONE parameter at a time builds intuition

I do NOT need to memorize values.
I need to understand behavior.


#### **✅ Topic 7 Summary**

Today I practiced:
- Isolating parameters
- Observing behavior changes
- Building real intuition

This skill directly translates to:
- debugging hallucinations
- tuning chatbots
- controlling production systems

#### **# 🧪 Mini Mock Test — LLM Parameters**


#### **Q1️⃣ What problem does `temperature` solve in LLMs?**

#### **A1:** Temperature controls randomness in token selection, balancing creativity vs determinism.
---

#### **Q2️⃣ Difference between `temperature` and `top_p` in one sentence.**

#### **A2:** Temperature controls randomness; top_p controls how many high-probability tokens are considered.
---

##### **Q3️⃣ Why is not setting `max_tokens` dangerous in production?**

#### **A3:** It can lead to runaway costs, long responses, latency spikes, and unsafe outputs.
---

##### **Q4️⃣ When is `stop` sequence mandatory?**

#### **A4:** When returning structured output like JSON, SQL, tool arguments, or API responses.
---

#### **Q5️⃣ You are building an API that returns JSON.What parameter values would you choose and why?**

#### **A5:** Low temperature (0.0–0.3), low top_p (0.2–0.4), strict max_tokens, and stop sequences to prevent extra text.
---

#### **Q6️⃣ True or False:Using a low temperature alone is enough to prevent hallucinations.**

#### **A6:** False — hallucination control requires multiple parameters + grounding + guardrails.
---

#### **Q7️⃣ Why do senior engineers think in parameter *combinations* instead of single parameters?**

#### **A7:** Because real behavior emerges from how parameters interact, not from isolated values.
---

#### **Q8️⃣ Interview question:“How do you control LLM behavior in production?”**

#### **A8:** “By tuning temperature, top_p, max_tokens, and stop sequences together based on the use case, with validation and guardrails.”